In [1]:
using Pkg; Pkg.activate(".")
using Onion

  Activating project at `c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer`


dev env


In [2]:
using Flux, DLProteinFormats, Onion, RandomFeatureMaps, StatsBase, Plots

In [3]:
dat = DLProteinFormats.load(PDBSimpleFlat500);

L = 30
train_inds = findall(dat.len .> L)
 
function random_batch(dat, L, B, filt_inds)
    locs = zeros(Float32, 3, L, B)
    inds = sample(filt_inds, B, replace=false)
    AAs = zeros(Int, L, B)
    for (i,ind) in enumerate(inds)
        l_range = rand(1:dat[ind].len - L + 1)
        locs[:, :, i] = dat[ind].locs[:, 1, l_range:l_range+L-1]
        AAs[:, i] = dat[ind].AAs[l_range:l_range+L-1]
    end
    return (;locs, AAs = Flux.onehotbatch(AAs, 1:20))
end

batch = random_batch(dat, L, 10, train_inds);
 

(locs = Float32[-3.9385026 -3.6966026 … -3.5770035 -3.7612357; 0.65176547 0.5978653 … 1.1975311 1.1599976; 3.0131392 3.2401397 … 2.994072 3.275505;;; 0.12875375 0.4219207 … -0.34287938 -0.20107937; -2.2870686 -2.3451352 … 0.035398103 -0.1321352; 2.0883386 2.185972 … 3.2084725 3.4243722;;; 0.091526784 0.3406601 … -0.39400655 -0.5211066; 2.3816497 2.26275 … 0.7014165 0.4277831; -3.2392564 -3.1221561 … 1.67681 1.8263104;;; 1.4990135 1.3195808 … 0.5398796 0.57967985; 1.0831373 0.9486038 … 0.47767067 0.74590415; 2.5403047 2.7598388 … 0.8273743 0.9601685;;; -0.5998432 -0.32047692 … 1.5906563 1.664257; -0.8861164 -0.87424964 … -0.017616654 -0.05158348; 0.010995102 0.20426139 … 0.5110611 0.21082763;;; 0.9618603 0.81229365 … 1.0263271 0.9560272; 0.42254257 0.68144286 … 0.8006094 0.6331093; -2.4700665 -2.5526001 … -1.4997667 -1.2546667;;; -1.0587637 -1.3507638 … 0.694703 0.6891364; 1.247051 1.0824511 … 0.7077844 0.8392844; -0.23291817 -0.2066513 … 0.62244856 0.9040815;;; -0.09710542 0.18332791 …

In [4]:
struct Toy1{L}
    layers::L
end
Flux.@layer Toy1
function Toy1(dim, depth)
    layers = (;
        loc_encoder = Dense(3 => dim, bias=false),
        transformers = [Onion.TransformerBlock(dim, 8, rope=Onion.MultiDimRoPE) for _ in 1:depth],
        AA_decoder = Dense(dim => 20, bias=false),
    )
    return Toy1(layers)
end
function (m::Toy1)(locs)
    l = m.layers
    x = l.loc_encoder(locs)
    for transformerblock in l.transformers
        x = transformerblock(x, 0, nothing, locs)
        #locs = updatelocs(x, locs)
    end
    aa_logits = l.AA_decoder(x)
    return aa_logits
end

In [5]:
model = Toy1(64, 4)
opt_state = Flux.setup(AdamW(eta = 0.001), model)
losses = Float32[]
model.layers.transformers[1]

MethodError: MethodError: no method matching Attention(::Int64, ::Int64, ::Int64; qkv_bias::Bool, rope::UnionAll)
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  Attention(::Int64, ::Int64, ::Any; qkv_bias) got unsupported keyword argument "rope"
   @ Onion c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\Onion.jl\src\GQAttention.jl:70
  Attention(::DA, ::DB, ::DC, !Matched::DD, !Matched::Int64, !Matched::Int64, !Matched::Int64, !Matched::Int64) where {DA, DB, DC, DD} got unsupported keyword arguments "qkv_bias", "rope"
   @ Onion c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\Onion.jl\src\GQAttention.jl:58
  Attention(::Int64, ::Int64; ...)
   @ Onion c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\Onion.jl\src\GQAttention.jl:70


In [ ]:
test = random_batch(dat, L, 10, train_inds)

(locs = Float32[-1.0946 -0.8099 … -0.7608333 -0.7133666; 1.9101667 1.7437668 … 2.7523334 2.5680668; -0.7178068 -0.70284003 … 0.10532646 -0.13450699;;; 1.0865543 1.2063221 … -1.0642449 -0.9757118; -0.0429677 -0.17960104 … -0.89093435 -1.207001; 0.58967286 0.8564056 … -0.99806136 -0.97466123;;; -2.2441115 -2.283478 … -1.7469113 -1.6522777; -1.9385716 -1.6220715 … -1.7053382 -1.7297716; -0.26328963 -0.2874565 … -0.119456485 0.20194359;;; -2.192054 -2.120454 … -1.2914875 -1.4886874; 1.5061543 1.2057877 … -0.46401232 -0.41351232; 1.8235438 1.8797104 … 3.2378106 3.4923768;;; 3.348346 3.1180465 … 0.50247955 0.65818024; -0.851297 -0.95419616 … -0.16626282 -0.29263002; -2.33311 -2.50401 … -3.6403441 -3.4063447;;; -1.0151192 -1.061886 … -0.08075257 -0.1506525; -0.20910454 -0.41390437 … -0.8461712 -0.5103712; -0.82430065 -0.59916735 … 1.1374992 1.0903994;;; -0.63200074 -0.7567673 … 0.42483273 0.42863265; 0.33595055 0.15718393 … -0.42574936 -0.70101607; -1.2933768 -1.0786768 … -2.4291434 -2.290977

In [7]:
model.layers.transformers[1].attention

UndefVarError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing BioStructures in the current active module Main

In [8]:
model(test[1].locs)

UndefVarError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing BioStructures in the current active module Main